In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# Dummy tokenizer (replace with real tokenizer in practice)
def simple_tokenizer(text):
    return text.lower().split()

# Vocabulary (for example)
vocab = {'<pad>': 0, '<unk>':1, 'this':2, 'is':3, 'good':4, 'bad':5, 'okay':6}
vocab_size = len(vocab)

def encode(text):
    tokens = simple_tokenizer(text)
    return [vocab.get(token, vocab['<unk>']) for token in tokens]

# Dataset class
class TextDataset(Dataset):
    def __init__(self, texts, labels, max_len=10):
        self.texts = texts
        self.labels = labels
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        encoded = encode(self.texts[idx])
        # Padding/truncation
        if len(encoded) < self.max_len:
            encoded += [vocab['<pad>']] * (self.max_len - len(encoded))
        else:
            encoded = encoded[:self.max_len]
        return torch.tensor(encoded), torch.tensor(self.labels[idx])

# Simple LSTM classifier
class LSTMClassifier(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, num_classes):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size, padding_idx=0)
        self.lstm = nn.LSTM(embed_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        emb = self.embedding(x)
        _, (hn, _) = self.lstm(emb)
        out = self.fc(hn[-1])
        return out

# Example data
texts = ["This is good", "This is bad", "This is okay"]
labels = [0, 1, 2]  # 0=positive,1=negative,2=neutral

dataset = TextDataset(texts, labels)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

# Model, loss, optimizer
model = LSTMClassifier(vocab_size, embed_size=8, hidden_size=16, num_classes=3)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Training loop (few epochs just for demonstration)
for epoch in range(10):
    for batch_x, batch_y in dataloader:
        optimizer.zero_grad()
        outputs = model(batch_x)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1} Loss: {loss.item():.4f}")

# Testing on a new sentence
def predict(text):
    model.eval()
    encoded = encode(text)
    if len(encoded) < 10:
        encoded += [vocab['<pad>']] * (10 - len(encoded))
    else:
        encoded = encoded[:10]
    input_tensor = torch.tensor(encoded).unsqueeze(0)  # batch size 1
    with torch.no_grad():
        output = model(input_tensor)
        pred = torch.argmax(output, dim=1).item()
    labels_map = {0: "Positive", 1: "Negative", 2: "Neutral"}
    return labels_map[pred]

print(predict("This is good"))  # Should predict Positive
print(predict("This is bad"))   # Should predict Negative
print(predict("This is okay"))  # Should predict Neutral


Epoch 1 Loss: 1.2619
Epoch 2 Loss: 1.0928
Epoch 3 Loss: 1.2072
Epoch 4 Loss: 1.0549
Epoch 5 Loss: 1.0983
Epoch 6 Loss: 1.1068
Epoch 7 Loss: 1.0915
Epoch 8 Loss: 1.0546
Epoch 9 Loss: 1.1520
Epoch 10 Loss: 1.1324
Positive
Neutral
Neutral


In [4]:
import torch
import torch.nn as nn

# --- Minimal vocab and tokenizer ---
vocab = {'<pad>': 0, '<unk>':1, 'this':2, 'is':3, 'good':4, 'bad':5, 'okay':6}
vocab_size = len(vocab)

def simple_tokenizer(text):
    return text.lower().split()

def encode(text, max_len=10):
    tokens = simple_tokenizer(text)
    encoded = [vocab.get(t, vocab['<unk>']) for t in tokens]
    if len(encoded) < max_len:
        encoded += [vocab['<pad>']] * (max_len - len(encoded))
    else:
        encoded = encoded[:max_len]
    return encoded

# --- Model Definition (same as before) ---
class LSTMClassifier(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, num_classes):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size, padding_idx=0)
        self.lstm = nn.LSTM(embed_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        emb = self.embedding(x)
        _, (hn, _) = self.lstm(emb)
        out = self.fc(hn[-1])
        return out

# --- Instantiate and (optionally) load pretrained weights ---
model = LSTMClassifier(vocab_size, embed_size=8, hidden_size=16, num_classes=3)
# Normally, load pretrained weights here if you have them:
# model.load_state_dict(torch.load('model.pth'))

model.eval()

# --- Get user input and predict ---
text = input("Enter a sentence to classify (positive/negative/neutral): ")
encoded = encode(text)
input_tensor = torch.tensor(encoded).unsqueeze(0)  # batch size 1

with torch.no_grad():
    output = model(input_tensor)
    pred = torch.argmax(output, dim=1).item()

labels_map = {0: "Positive", 1: "Negative", 2: "Neutral"}
print(f"Predicted sentiment: {labels_map[pred]}")


Enter a sentence to classify (positive/negative/neutral):  i love it


Predicted sentiment: Positive
